In [46]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models

# ResNet18 with custom head and loss

In [22]:
resnet18 = models.resnet18(pretrained=True)

In [23]:
m = resnet18
m.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
m(torch.randn(1,3,28,28)).shape

torch.Size([1, 1000])

In [59]:
class H(nn.Module): 
    def __init__(self, i_d):
        super(H, self).__init__()
        self.i_d = i_d
        self.l   = nn.Linear(1000, 10)
        self.ll  = nn.Linear(4, 4)
        self.lr  = nn.Linear(6, 6)
    def forward(self, x):
        a1 = self.l(x)
        a1 = torch.squeeze(a1)
        al = self.ll(a1[:4])
        ar = self.lr(a1[4:])
        return torch.cat((
            F.softmax(al, dim=0),
            F.softmax(ar, dim=0)
        ))

In [60]:
i_d = 1000
h = H(i_d)
h(torch.randn((1000)))

tensor([0.5846, 0.0968, 0.1182, 0.2004, 0.0992, 0.2283, 0.1878, 0.1996, 0.1520,
        0.1329], grad_fn=<CatBackward>)

In [112]:
y = h(m(torch.randn(1,3,28,28)))
y

tensor([0.4740, 0.1445, 0.0963, 0.2852, 0.0861, 0.2118, 0.2681, 0.1979, 0.1388,
        0.0973], grad_fn=<CatBackward>)

In [93]:
target1 = torch.Tensor([2]).type(torch.long)
target2 = torch.Tensor([3]).type(torch.long)

In [106]:
def loss(input, t1, t2):
    i1 = torch.unsqueeze(input[:4], dim=0)
    i2 = torch.unsqueeze(input[4:], dim=0)
    l1 = F.cross_entropy(i1, t1)
    l2 = F.cross_entropy(i2, t2)
    return l1 + l2

In [109]:
l = loss(y, target1, target2)

In [110]:
l.backward()